In [41]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [49]:
loader = TextLoader('Woodcraft and Camping.txt', encoding='utf-8')
documents = loader.load()

In [50]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [51]:
len(texts)

3408

In [52]:
texts[10]

Document(page_content='Distributed Proofreading Team at http://www.pgdp.net', metadata={'source': 'Woodcraft and Camping.txt'})

In [53]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [54]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [55]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 9.8 s
Wall time: 13.1 s


In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("Woodcraft and Camping.txt')")

In [ ]:
len(docs)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [63]:
query = "What do we do if we get lost?"
process_llm_response(qa_chain, query)

Query: What do we do if we get lost?

Inference time: 0.488 sec.

Result:  If we get lost, we should stop and look around to see if we can find any landmarks or signs that might help us figure out where we are. If we still can't find our way, we should try to retrace our steps and follow the route we came in on. If that doesn't work, we should try to find a nearby town or city and ask for directions. It's always better to be safe than sorry when it comes to getting lost, so we should never hesitate to ask for help if we need it.

metadata: {'source': 'The existence of Gods.txt'}


In [64]:
query = "What is the importance of making the camp-fire?"
process_llm_response(qa_chain, query)

Query: What is the importance of making the camp-fire?

Inference time: 5.618 sec.

Result:  The camp-fire is important because it provides warmth and light for the camp, and it can also be used for cooking and boiling water for drinking. Making a proper camp-fire is crucial to ensure safety and prevent accidents. It should be built in a clear area away from flammable materials, and proper precautions should be taken to prevent it from spreading. Additionally, the fire should be properly extinguished before leaving the campsite to prevent wildfires.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [65]:
query = "List out 5 things we need to bring for camping"
process_llm_response(qa_chain, query)

Query: List out 5 things we need to bring for camping

Inference time: 3.059 sec.

Result:  Some things we might need to bring for camping include a tent, sleeping bags, a camping stove for cooking, a cooler for storing food, and a first aid kit in case of any accidents. Do you have any other suggestions?

metadata: {'source': 'Woodcraft and Camping.txt'}


In [67]:
query = "How camp cooking is usually done? "
process_llm_response(qa_chain, query)

Query: How camp cooking is usually done? 

Inference time: 8.185 sec.

Result:  Camp cooking typically involves cooking meals outdoors using portable stoves, grills, or open fires. Food is often prepared in advance and packed in coolers or insulated bags to keep it fresh. Simple ingredients and easy-to-prepare meals are commonly used to minimize the amount of equipment and cleanup required. Some common camping foods include hot dogs, hamburgers, foil packet meals, and pre-packaged camping meals.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [68]:
query = "2 Simple receipts for camp cooking"
process_llm_response(qa_chain, query)
#1st time Different answer

Query: 2 Simple receipts for camp cooking

Inference time: 16.101 sec.

Result:  1. Campfire Bannock:
Ingredients:
- 2 cups all-purpose flour
- 1 tbsp baking powder
- 1 tsp salt
- 1/4 cup vegetable oil
- 3/4 cup water

Instructions:
1. In a large bowl, mix together the flour, baking powder, and salt.
2. Add the vegetable oil and water, and stir until a dough forms.
3. Knead the dough on a floured surface for 1-2 minutes.
4. Roll the dough out into a circle about 1/4 inch thick.
5. Place the circle directly on the campfire grate or a campfire-safe skillet.
6. Cook for 2-3 minutes on each side, or until golden brown.

2. Campfire Pancakes:
Ingredients:
- 1 cup all-purpose flour
- 2 tbsp sugar
- 2 tsp baking powder
- 1/2 tsp salt
- 1 cup milk
- 1 egg
- 2 tbsp vegetable oil

Instructions:
1. In a large bowl, mix together the flour, sugar, baking powder, and salt.
2. Add the milk, egg, and vegetable oil, and stir until just combined.
3. Heat a campfire-safe skillet over the campfire.
4. Pou

In [69]:
query = "Reasons for Preferring the Clinker-Built Cedar?"
process_llm_response(qa_chain, query)

Query: Reasons for Preferring the Clinker-Built Cedar?

Inference time: 4.31 sec.

Result:  The author prefers the clinker-built cedar because it is light, strong, and durable, and it can be built into any part of the ground that the builder pleases. The author also mentions that some people still prefer this type of boat despite its perceived flaws, such as being "merciful, good, wise, faithful." However, the author does not provide specific reasons for these preferences.

metadata: {'source': 'The existence of Gods.txt'}


In [70]:
query = "How to make a camp?"
process_llm_response(qa_chain, query)

Query: How to make a camp?

Inference time: 7.151 sec.

Result:  To make a camp, you will need a tent, a sleeping bag, a camping stove, a cooler for food, a lantern, and a camping chair. First, find a level spot to pitch your tent. Clear the area of any debris and stakes. Assemble the tent poles and insert them into the tent sleeves. Spread the tent fly over the tent and secure it with the tent stakes. Lay out your sleeping bag inside the tent. Set up your camping stove and cooler nearby. Hang your lantern from a tree branch or stake it into the ground. Place your camping chair in a comfortable spot to relax and enjoy the outdoors. Remember to pack out all trash and leave the campsite as you found it.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [71]:
query = "The right way to make a camp fire?"
process_llm_response(qa_chain, query)
#Although can answer total number of receipes contain, still not able to answer this one

Query: The right way to make a camp fire?

Inference time: 2.175 sec.

Result:  According to the illustration, the right way to make a camp fire is shown in the illustration. The wrong way to make a camp fire is not explicitly stated in the context provided.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [73]:
query = "Where to go for an Outing?"
process_llm_response(qa_chain, query)

Query: Where to go for an Outing?

Inference time: 4.825 sec.

Result:  If you're looking for a place to go for an outing, consider visiting the local nature preserve. Not only will you be able to enjoy the beautiful scenery, but you'll also be supporting a worthy cause through your admission fee and any donations you make. Plus, many nature preserves offer educational programs and guided tours, making it a great learning opportunity for all ages. So why not make a difference while having fun? Head to your nearest nature preserve today!

metadata: {'source': 'New Vegetarian Dishes.txt'}


In [80]:
query = "Give me an example of a easily made and very comfortable camp"
process_llm_response(qa_chain, query)
#Finally can answer this question

Query: Give me an example of a easily made and very comfortable camp

Inference time: 5.764 sec.

Result:  A winter camp could be an example of a easily made and very comfortable camp. In winter, people can build a camp by digging a pit in the snow, lining it with branches and leaves, and covering it with more snow. This type of camp provides insulation and keeps the occupants warm. They can also build a fire inside the camp for additional warmth. Overall, a winter camp can be a cozy and comfortable shelter in harsh winter conditions.

metadata: {'source': 'Woodcraft and Camping.txt'}


In [23]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x28241343510>)

In [24]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
